## Live Predictions for given input data

#### model takes required params in required format. input columns list looks like this:
#####                  ["uprn", "latitude", "longitude", "property_type", "number_of_bedrooms"]


#### after finishing running, it generates output data for individual property including time series (truth and forecast values and normalized values as well . outpul columns list looks like this: 
#####                ["uprn","property_type", 'pred', 'area_code', 'district_code' ,'last_transaction_price', 'last_transaction_date'," target_ts", "forecast",  "normalized_target_ts","normalized_forecast"])

In [269]:
# import libs
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import h3
from prophet import Prophet
from copy import deepcopy
from tqdm import tqdm
import os
import datetime 
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from IPython.display import clear_output
# import catboost as cb
from awswrangler.athena import read_sql_query
import quantstats as qs

### Here you can change date parameters

In [116]:
TARGET_TIME = "2022-04-01"
TARGET_TIME_12m = "2022-04-01"
TARGET_TIME_24m = "2023-04-01"
TRAINING_MONTHS = 48
PREDICTION_MONTHS = 24+1
LAST_TRANSACTION_DATE_LOWER_BOUND = datetime.datetime.strptime('2019-01-01', "%Y-%m-%d").date()

## Training data from S3

In [ ]:
source_price_asked = pd.read_parquet("s3://francesco-magic-corner/predictive_model/comparables/listings/")
source_price_asked = source_price_asked.loc[source_price_asked["date"]>pd.to_datetime("2012-01-01")].reset_index(drop=True)
source_price_asked["bedrooms"] = source_price_asked["rooms"]

## input data with required params

#### input data format is strictly determined, in case of mismatch, error message is returned

In [168]:
required_columns = ["uprn", "latitude", "longitude", "property_type", "number_of_bedrooms"]
input_df = pd.read_csv("input_data.csv").fillna('')
input_columns = input_df.columns
for col in required_columns:
    if col not in input_columns:
        raise ValueError('incorrect input format')

## Merging input data with data stored in Athena tables

#### here we aim to get last transaction information from our db for performing calculations

In [198]:
merged_df = input_df
dfs_list = []
uprns = input_df["uprn"]
for uprn in uprns:
    query = (
            f"SELECT uprn,latitude,longitude,area_code,district_code,postcode,\
            number_habitable_rooms,price,date_of_transfer, property_type FROM lr_epc_matched WHERE uprn = {uprn}"
        )
    dfs = read_sql_query(
        sql=query,
        database="realyse-data-pipelines-master-builds",
        ctas_approach=True,
        chunksize=100_000,
        max_cache_seconds=86400,
    )
    for df in dfs:
        if not isinstance(df, pd.DataFrame):
            continue
        dfs_list.append(df)
    
    
# uprns  

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [280]:
res_df = dfs_list[0]
for i in range (len(dfs_list)):
    if i == 0:
        continue
    res_df = pd.concat([res_df,dfs_list[i]])

In [281]:
res_df = res_df[res_df["date_of_transfer"]>=datetime.datetime.strptime('2019-01-01', "%Y-%m-%d").date()]
res_df = res_df.rename(columns={'price': 'last_transaction_price', 'date_of_transfer': 'last_transaction_date'})
res_df = res_df[res_df["last_transaction_date"]>=LAST_TRANSACTION_DATE_LOWER_BOUND]
res_df = pd.merge(res_df, input_df, how='inner',left_on=['uprn','property_type'],right_on=['uprn','property_type' ])
res_df = res_df.drop(["latitude_y", "longitude_y"],axis=1)
merged_df = res_df.rename(columns={'latitude_x': 'latitude', 'longitude_x': 'longitude'})
merged_df["pred"] = merged_df["number_of_bedrooms"]
# merged_df

## class for suppressing stdout 

#### the class is responsible for avoiding printing errors

In [282]:
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

## class for reference property

In [283]:
class ReferenceProperty():
    
    def __init__(
        self, 
        area:str,
        latitude: float, 
        longitude: float, 
        property_type: str, 
        target_time: str, 
        target_feature: str, 
        training_months: int,
        prediction_months: int,
        nbedrooms: int = None, 
        nrooms: int = None, 
    ):
        
        """
        Initialisation
        """
        
        # input information regarding the reference property
        self.latitude = latitude
        self.longitude = longitude
        self.property_type = property_type
        # both bedrooms and rooms (ie, number of habitable rooms is supported)
        self.nbedrooms = nbedrooms        
        self.nrooms = nrooms
        # final data for training
        self.target_time = pd.to_datetime(target_time)
        # target feature on which the time series is built (usually price)
        self.target_feature = target_feature
        # how many months to train on
        self.training_months = training_months
        # how much in the future to predict
        self.prediction_months = prediction_months
        # how far away from the central hexagon to look at
        self.neighboring_radius = 5
        self.area = area
        
        # variable used to store the output
        # value of target variable at target time
        self.target_mean = None
        # std of target variable at target time
        self.target_dev = None
        # sample of properties in the area at target time
        self.sample = None
        # time series of values and stds
        self.target_ts = None
        
        # value of target variable at target time
        self.target_means = []
        # std of target variable at target time
        self.target_devs = []
        # sample of properties in the area at target time
        self.samples = []
        # time series of values and stds
        self.target_tss = []
    
    def compute_sales_ts(self, sales: pd.DataFrame):
        
        """
        Method to compute the time series of target value
        """
        
        # compute distance to ref index and define a weight which inversly proportional to the distance from the reference coordinates
        sales[f"hex_{self.resolution}_distance"] = 1./(sales[f"hex_{self.resolution}"].apply(lambda x: 2*h3.h3_distance(x, self.h3_index))+1)
        # sales[f"hex_{self.resolution}_distance"] = 1.
        
        # time series of target
        # output initialisation
        target_ts = []
        target_ts_vals = []
        target_ts_stds = []
        sample = None
        target_mean = None
        target_std = None
        
        # looping on months to build time series
        for idx, month in enumerate(list(range(-self.training_months, self.prediction_months))):
            
            # rolling period of time
            time = pd.to_datetime(self.target_time + relativedelta(months=+month))
            time_m = pd.to_datetime(time - relativedelta(months=+6))
            
            # select sales in the given period
            sales1 = sales.loc[
                (sales["date"]<=time)
                &(sales["date"]>=time_m)
            ]
            
            # return if no sales and we are in the training period
            if len(sales1)<5 and month<=0:
                return None, None, None, None
            # removing outliers (THIS SHOULD BE DONE IN A BETTER WAY, AT THE MOMENT EVERYTHING ABOVE 2 SIGMA IS REMOVED)
            mean_s = (sales1[f"hex_{self.resolution}_distance"]*sales1[self.target_feature]).sum()/(sales1[f"hex_{self.resolution}_distance"].sum()+0.001)
            std_s = ((sales1[f"hex_{self.resolution}_distance"]*((sales1[self.target_feature] - mean_s)**2)).sum()/(sales1[f"hex_{self.resolution}_distance"].sum()+0.001))**0.5
            sales1 = sales1.loc[
                abs(sales1[self.target_feature]-mean_s)<2*std_s
            ]
            
            # return if no sales and we are in the training period
            if len(sales1)<5 and month<=0:
                return None, None, None, None
            
            # recompute adjusted mean and stds
            mean_s = (sales1[f"hex_{self.resolution}_distance"]*sales1[self.target_feature]).sum()/(sales1[f"hex_{self.resolution}_distance"].sum()+0.001)
            std_s = ((sales1[f"hex_{self.resolution}_distance"]*((sales1[self.target_feature] - mean_s)**2)).sum()/(sales1[f"hex_{self.resolution}_distance"].sum()+0.001))**0.5
            target_ts.append({"start":time_m,"end":time})
            
            # save weighted avg and std
            target_ts_vals.append(mean_s)
            target_ts_stds.append(std_s)
            
            # month 0 is current date so we save it separatly
            if month==0:
                target_mean = mean_s
                target_std = std_s
                sample = sales1
        
        # append to output the relative index of values
        for i,t in enumerate(target_ts):
            target_ts[i].update({"value": target_ts_vals[i]/target_mean})
            target_ts[i].update({"value_std": target_ts_stds[i]/target_mean})
        
        # creating output dataframe
        target_ts = pd.DataFrame(target_ts)
        target_ts = target_ts.sort_values("start")
                
        return sample, target_mean, target_std, target_ts

    def fit(
        self, 
        sales_listings: pd.DataFrame, 
    ):
        
        """
        Method to create the reference property given coordinates and transactions
        """
        radius_upper_bound = 25
        resolutions = [10, 9, 8]
        lt_prices_residuals_abs=[]
        samples=[]
        target_means=[]
        target_devs=[]
        target_tss = []
        while (self.neighboring_radius < radius_upper_bound):
            # define which resolution to use
            for i in range (0, 3):
                # define area filter usage
                for j in range (0,2):
                    self.resolution = resolutions[i]
                    # compute the index of the reference property given the resolution
                    self.h3_index = h3.geo_to_h3(lat=self.latitude, lng=self.longitude, resolution=self.resolution)
                    # compute the neighboring indices
                    self.neighboring_indices = h3.k_ring(self.h3_index,self.neighboring_radius)

                    # filter out given the parameters above
                    if self.nbedrooms:
                        sales_listing_copy = sales_listings.loc[
                                (sales_listings["bedrooms"]==self.nbedrooms)
                                &(sales_listings["property_type"]==self.property_type)
                                &(sales_listings[self.target_feature]>0)
                                &(sales_listings[f"hex_{self.resolution}"].isin(self.neighboring_indices))
                                &((j==0)|(sales_listings["area"]==self.area))
                        ]
                    else:
                        sales_listing_copy = sales_listings.loc[
                                (sales_listings["rooms"]==self.nrooms)
                                &(sales_listings["property_type"]==self.property_type)
                                &(sales_listings[self.target_feature]>0)
                                &(sales_listings[f"hex_{self.resolution}"].isin(self.neighboring_indices))
                                &((j==0)|(sales_listings["area"]==self.area))
                        ]

                    # compute time series for target variable 
                    self.sample, self.target_mean, self.target_dev, self.target_ts =  self.compute_sales_ts(sales_listing_copy)
                    if self.sample is not None:
                        self.samples.append(self.sample)
                        self.target_means.append(self.target_mean)
                        self.target_devs.append(self.target_dev)
                        self.target_tss.append(self.target_ts)

            
            self.neighboring_radius *=1.5
            
        
        

In [286]:
# looping over all  properties
ref_props = []

for idx,row in tqdm(merged_df.iterrows(),total=len(merged_df)):
    # define what to look for
    LATITUDE = row["latitude"]
    LONGITUDE = row["longitude"]
    BEDROOMS = row["pred"]    
    PROPTYPE = row["property_type"]
    TARGET_FEATURE = "price"
    UPRN = row["uprn"]
    AREA = row["area_code"]
    lt_date = str(row["last_transaction_date"])
    lt_price = row["last_transaction_price"]
#     print(UPRN)
#     break
    # initialise class
    ref_prop = ReferenceProperty(
        area = AREA,
        latitude=LATITUDE, 
        longitude=LONGITUDE,
        property_type=PROPTYPE, 
        target_time=TARGET_TIME, 
        target_feature=TARGET_FEATURE, 
        training_months=TRAINING_MONTHS,
        prediction_months=PREDICTION_MONTHS,
        nbedrooms=BEDROOMS
#         nrooms=BEDROOMS,
    )

    # fit reference property
    ref_prop.fit(
        sales_listings=source_price_asked
    )

    # fit prophet model
    # resample dates to days
    if ref_prop.sample is None:
        ref_prop.forecast = None
        ref_props.append({"uprn": UPRN ,"rooms": BEDROOMS, "type": PROPTYPE, "latitude": LATITUDE, "longitude": LONGITUDE,  "ref": deepcopy(ref_prop), "lt_date": row["last_transaction_date"], "lt_price": row["last_transaction_price"], "pidx": idx})
    else:
        lt_prices_residuals = []
        for i in range (0, len(ref_prop.target_means)):
            tmp = ref_prop.target_ts.copy()

            # fill missing values with noisy entries with appropriate std dev
            date_range = pd.DataFrame(
                {'ds': tmp.loc[tmp["end"]<=ref_prop.target_time, "end"], "y": tmp.loc[tmp["end"]<=ref_prop.target_time, "value"]}
            )

            # create prophet model
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=False,
                daily_seasonality=False,
            )
            with suppress_stdout_stderr():
                m.fit(date_range)

            future = pd.DataFrame({'ds': pd.date_range(
                tmp.loc[tmp["end"]<=ref_prop.target_time, "end"].min(), 
                periods=TRAINING_MONTHS+PREDICTION_MONTHS+1, freq='M'
            ).date+relativedelta(days=+1)})
            forecast = m.predict(future)
            ltpi = forecast.loc[(forecast["ds"]<pd.to_datetime(lt_date)+relativedelta(months=+1))&(forecast["ds"]>=pd.to_datetime(lt_date)), "yhat"].iloc[0]
            ltpp = ref_prop.target_means[i]*ltpi
            ltpprp = round((ltpp-lt_price)/lt_price,3)
            lt_prices_residuals.append(ltpprp)
        lt_prices_residuals = np.abs(lt_prices_residuals)
        min_value = min(lt_prices_residuals)
        min_index=np.where(lt_prices_residuals == min_value)[0][0]
        ref_prop.sample = ref_prop.samples[min_index]
        ref_prop.target_mean = ref_prop.target_means[min_index]
        ref_prop.target_dev = ref_prop.target_devs[min_index]
        ref_prop.target_ts = ref_prop.target_tss[min_index]
        
        
        
        tmp = ref_prop.target_ts.copy()

        # fill missing values with noisy entries with appropriate std dev
        date_range = pd.DataFrame(
            {'ds': tmp.loc[tmp["end"]<=ref_prop.target_time, "end"], "y": tmp.loc[tmp["end"]<=ref_prop.target_time, "value"]}
        )

        # create prophet model
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
        )
        with suppress_stdout_stderr():
            m.fit(date_range)

        future = pd.DataFrame({'ds': pd.date_range(
            tmp.loc[tmp["end"]<=ref_prop.target_time, "end"].min(), 
            periods=TRAINING_MONTHS+PREDICTION_MONTHS+1, freq='M'
        ).date+relativedelta(days=+1)})
        forecast = m.predict(future)
        forecast["yhat_upper_std"] = forecast["yhat"] + ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value_std"].iloc[0]
        forecast["yhat_lower_std"] = forecast["yhat"] - ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value_std"].iloc[0]
        forecast["yhat_upper_total"] = forecast["yhat"] + np.sqrt((forecast["yhat_upper"]-forecast["yhat"])**2 + ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value_std"].iloc[0]**2)
        forecast["yhat_lower_total"] = forecast["yhat"] - np.sqrt((forecast["yhat_lower"]-forecast["yhat"])**2 + ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value_std"].iloc[0]**2)  
        with suppress_stdout_stderr():
        
            # compute metrics at 0, 12 and 24 months
            ap0 = ref_prop.target_mean*ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value"].iloc[0] 
            ai0 = ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value"].iloc[0] 
            pi0 = forecast.loc[forecast["ds"]==TARGET_TIME, "yhat"].iloc[0] 
            pi0e = forecast.loc[forecast["ds"]==TARGET_TIME, "yhat_upper"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME, "yhat"].iloc[0] 
            pi0s = forecast.loc[forecast["ds"]==TARGET_TIME, "yhat_upper_std"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME, "yhat"].iloc[0] 
            pi0t = forecast.loc[forecast["ds"]==TARGET_TIME, "yhat_upper_total"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME, "yhat"].iloc[0] 
            pp0 = ref_prop.target_mean*pi0
            pp0e = ref_prop.target_mean*pi0e
            pp0s = ref_prop.target_mean*pi0s
            pp0t = ref_prop.target_mean*pi0t
            e0 = (
                forecast.loc[forecast["ds"]==TARGET_TIME, "yhat"].iloc[0] 
                - ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value"].iloc[0]
            )/ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME, "value"].iloc[0]

            ap12 = ref_prop.target_mean*ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_12m, "value"].iloc[0] 
            ai12 = ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_12m, "value"].iloc[0] 
            pi12 = forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat"].iloc[0] 
            pi12e = forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat_upper"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat"].iloc[0]
            pi12s = forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat_upper_std"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat"].iloc[0] 
            pi12t = forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat_upper_total"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat"].iloc[0]
            pp12 = ref_prop.target_mean*pi12
            pp12e = ref_prop.target_mean*pi12e
            pp12s = ref_prop.target_mean*pi12s
            pp12t = ref_prop.target_mean*pi12t
            e12 = (
                forecast.loc[forecast["ds"]==TARGET_TIME_12m, "yhat"].iloc[0] 
                - ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_12m, "value"].iloc[0]
            )/ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_12m, "value"].iloc[0]

            ap24 = ref_prop.target_mean*ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_24m, "value"].iloc[0] 
            ai24 = ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_24m, "value"].iloc[0] 
            pi24 = forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat"].iloc[0] 
            pi24e = forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat_upper"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat"].iloc[0]
            pi24s = forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat_upper_std"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat"].iloc[0] 
            pi24t = forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat_upper_total"].iloc[0] - forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat"].iloc[0]
            pp24 = ref_prop.target_mean*pi24
            pp24e = ref_prop.target_mean*pi24e
            pp24s = ref_prop.target_mean*pi24s
            pp24t = ref_prop.target_mean*pi24t
            e24 = (
                forecast.loc[forecast["ds"]==TARGET_TIME_24m, "yhat"].iloc[0] 
                - ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_24m, "value"].iloc[0]
            )/ref_prop.target_ts.loc[ref_prop.target_ts["end"]==TARGET_TIME_24m, "value"].iloc[0]

            try:
                ltpi = forecast.loc[(forecast["ds"]<pd.to_datetime(row["last_transaction_date"])+relativedelta(months=+1))&(forecast["ds"]>=pd.to_datetime(row["last_transaction_date"])), "yhat"].iloc[0]
                ltpie = forecast.loc[(forecast["ds"]<pd.to_datetime(row["last_transaction_date"])+relativedelta(months=+1))&(forecast["ds"]>=pd.to_datetime(row["last_transaction_date"])), "yhat_upper"].iloc[0]-ltpi
                ltpis = forecast.loc[(forecast["ds"]<pd.to_datetime(row["last_transaction_date"])+relativedelta(months=+1))&(forecast["ds"]>=pd.to_datetime(row["last_transaction_date"])), "yhat_upper_std"].iloc[0]-ltpi
                ltpit = forecast.loc[(forecast["ds"]<pd.to_datetime(row["last_transaction_date"])+relativedelta(months=+1))&(forecast["ds"]>=pd.to_datetime(row["last_transaction_date"])), "yhat_upper_total"].iloc[0]-ltpi
                ltpp = ref_prop.target_mean*ltpi
                ltppe = ref_prop.target_mean*ltpie
                ltpps = ref_prop.target_mean*ltpis
                ltppt = ref_prop.target_mean*ltpit
            except:
                ltpi = np.nan
                ltpie = np.nan
                ltpis = np.nan
                ltpit = np.nan
                ltpp = np.nan
                ltppe = np.nan
                ltpps = np.nan
                ltppt = np.nan

            # saving values
            merged_df.loc[idx, "sample_size"] = len(ref_prop.sample)
            merged_df.loc[idx, "last_transaction_predicted_price"] = round(ltpp,3)
            merged_df.loc[idx, "last_transaction_index"] = round(row["last_transaction_price"]/ap0,3)
            merged_df.loc[idx, "last_transaction_predicted_price_model_error"] = round(ltppe,3)
            merged_df.loc[idx, "last_transaction_predicted_price_total_error"] = round(ltppt,3)
            merged_df.loc[idx, "last_transaction_predicted_price_residual_percentage"] = round((ltpp-row["last_transaction_price"])/row["last_transaction_price"],3)


            ref_prop.forecast = forecast
            ref_props.append({"uprn": UPRN ,"rooms": BEDROOMS, "type": PROPTYPE, "latitude": LATITUDE, "longitude": LONGITUDE,  "ref": deepcopy(ref_prop), "lt_date": row["last_transaction_date"], "lt_price": row["last_transaction_price"], "pidx": idx})



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]<ipython-input-286-ad2c8f0652a6>:147: RuntimeWarning: divide by zero encountered in double_scalars
  e24 = (
 50%|█████████████████████████████████████████▌                                         | 1/2 [01:56<01:56, 116.38s/it]<ipython-input-286-ad2c8f0652a6>:147: RuntimeWarning: divide by zero encountered in double_scalars
  e24 = (
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:58<00:00, 89.21s/it]


In [287]:
# pd.set_option('display.max_columns', 500)
# # # merged_df
# merged_df
    


In [288]:
ref_props_new = []
for i,p in enumerate(ref_props):
    forecast_dict={}
    normalized_forecast ={}
    target_ts_dict={}
    normalized_target_ts ={}
    ref_prop = p["ref"]
    if ref_prop.forecast is None:
        forecast = ref_prop.forecast
    else:
        for index, row in forecast.iterrows():
            forecast_dict[str(row["ds"].date())]=target_mean *row["yhat"]
#         normalized = (x-min(forecast_dict.values()))/(min(forecast_dict.values())-min(forecast_dict.values()))
        for key, value in forecast_dict.items():
            normalized_forecast[key] =(value-min(forecast_dict.values()))/(max(forecast_dict.values())-min(forecast_dict.values()))
    if ref_prop.target_ts is None:
        target_ts = ref_prop.target_ts
    else: 
        for index, row in target_ts.iterrows():
            target_ts_dict[str(row["end"].date())]=target_mean *row["value"]
        for key, value in target_ts_dict.items():
            normalized_target_ts[key] =(value-min(target_ts_dict.values()))/(max(target_ts_dict.values())-min(target_ts_dict.values()))
    
    if  ref_prop.forecast is not None and ref_prop.target_ts is not None : 
        ref_props_new.append({"uprn":p["uprn"] ,"target_mean": ref_prop.target_mean,"target_time" : ref_prop.target_time,"target_ts" : target_ts_dict,"normalized_target_ts" : normalized_target_ts,"forecast": forecast_dict,"normalized_forecast": normalized_forecast,"number_of_bedrooms": p["rooms"] , "type": p["type"], "latitude": p["latitude"], "longitude": p["longitude"], "lt_date": p["lt_date"], "lt_price": p["lt_price"], "pidx": i})

In [289]:
output = pd.merge(merged_df, pd.DataFrame(ref_props_new) , how='inner',left_on=['uprn','property_type', 'pred', 'last_transaction_price', 'last_transaction_date' ] ,right_on=['uprn','type', 'number_of_bedrooms', 'lt_price', 'lt_date' ])

In [293]:
result_columns = ["uprn","property_type", 'pred', 'area_code', 'district_code','last_transaction_price','last_transaction_date',"target_ts","forecast","normalized_target_ts","normalized_forecast"]
output = output[result_columns]
output = output.rename(columns={'pred': 'number_of_bedrooms'})
# output